In [1]:
from sgkit.distance.api import pairwise_distance_blocks, pairwise_distance_blockwise
import dask.array as da

from dask.distributed import Client, performance_report, get_task_stream
from dask.diagnostics import ProgressBar, ResourceProfiler
from bokeh.io import output_notebook

import numpy as np

In [2]:
!hostname

qgpu2


In [ ]:
from plotly.subplots import make_subplots
import plotly.express as px

from plotly import io as pio
pio.renderers.default = "png+plotly_mimetype"
# pio.orca.config.use_xvfb = True

In [3]:
def get_dask_cluster(n_workers=1):
    return Client(n_workers=n_workers)

## Data: size roughly 10% of MalariaGEN

In [11]:
def get_data():
    return da.random.choice(np.array([0, 1, 2], dtype='i1'), 
                         p=[.7, .2, .1,], 
                         size=(1142, 24767689//10), chunks=(50, 1000000))

In [5]:
# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=2)
# client
c = get_dask_cluster()
c

Client Scheduler: tcp://127.0.0.1:38069 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 48 Memory: 67.47 GB


In [6]:
def perf_test(pairwise_func, x):
    with ResourceProfiler() as prof:
        result = pairwise_func(x)
    return result, prof

In [12]:
x = get_data()
x

dask.array<da.random.choice, shape=(1142, 2476768), dtype=int8, chunksize=(50, 1000000), chunktype=numpy.ndarray>

## Pairwise without blockwise

In [14]:
%%time
blocks_result = perf_test(pairwise_distance_blocks, x)

CPU times: user 3min 11s, sys: 22.7 s, total: 3min 33s
Wall time: 6min 53s


## Pairwise with blockwise

In [13]:
%%time
blockwise_result = perf_test(pairwise_distance_blockwise, x)

/home/aktech/sgkit/sgkit/distance/api.py:154: PerformanceWarning: Increasing number of chunks by factor of 23
  out = da.blockwise(


CPU times: user 2min 47s, sys: 22.3 s, total: 3min 9s
Wall time: 6min 38s
